In [55]:
#importing libraries
import numpy as np
import tensorflow as tf
import helper

In [56]:
helper.download_dataset()
train_val_dataset, test_dataset = helper.load_images_labels()
train_val_images, train_val_labels = helper.extract_images_labels(train_val_dataset)
test_images, test_labels = helper.extract_images_labels(test_dataset)

from sklearn.model_selection import train_test_split
train_images, val_images, train_labels, val_labels = train_test_split(train_val_images, train_val_labels, test_size=0.2, random_state=42)

### Syntax:

1. placeholder: 
    - tf.placeholder(dtype, shape=None, name=None)
2. Variable: 
    - tf.Variable(initial_values, name=None)
    - tf.get_variable(name, shape=None, dtype=None)
3. initialize values: 
    - tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
    - tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
    - tf.zeros(shape, dtype=tf.float32, name=None)
    - tf.ones(shape, dtype=tf.float32, name=None)  
4. Mathematical Ops:
    - tf.add()
    - tf.matmul()
    - tf.multiply()
    - tf.log()
5. Activation functions:
    - tf.nn.relu(features, name=None)
    - sigmoid
    - tf.nn.softmax(logits, axis=None, name=None, dim=None)
    - tf.nn.softmax_cross_entropy_with_logits_v2(_sentinel=None, labels=None, logits=None, dim=-1, name=None)
6. Loss calculation:
    - tf.reduce_sum(input_tensor,  axis=None, keepdims=None, name=None, reduction_indices=None)
    - tf.reduce_mean(input_tensor, axis=None, keepdims=None, name=None, reduction_indices=None)
7. Optimization functions:
    - gradient descent
    - Adam optimizer
8. Session handling functions:
    - tf.initialize_all_variables()
    - tf.reset_default_graph()
9. Saving model:
    - tf.train.saver()


In [66]:
class NeuralNetwork:
    
    def __init__(self, num_inputs, num_outputs, num_hidden_layer1, num_hidden_layer2, learning_rate, epochs, batch_size):
        
        tf.reset_default_graph()
        
        self.inputs = tf.placeholder(tf.float32, shape=[None, num_inputs], name="inputs_placeholder")
        self.outputs = tf.placeholder(tf.float32, shape=[None, num_outputs], name="outputs_placeholder")
        
        self.weights_input_to_hidden = self.getWeights([num_inputs, num_hidden_layer1], var_name="weights_input_to_hidden")
        self.weights_hidden_to_hidden = self.getWeights([num_hidden_layer1, num_hidden_layer2], var_name="weights_hidden_to_hidden")
        self.weights_hidden_to_output = self.getWeights([num_hidden_layer2, num_outputs], var_name="weights_hidden_to_ouput")
        
        self.bias_hidden_layer1 = self.getBias([num_hidden_layer1], var_name="bias_hidden_layer1")
        self.bias_hidden_layer2 = self.getBias([num_hidden_layer2], var_name="bias_hidden_layer2")
        
        saver = tf.train.Saver({
            "weights_input_to_hidden" : self.weights_input_to_hidden,
            "weights_hidden_to_hidden" : self.weights_hidden_to_hidden,
            "weights_hidden_to_ouput" : self.weights_hidden_to_output,
            "bias_hidden_layer1" : self.bias_hidden_layer1,
            "bias_hidden_layer2" : self.bias_hidden_layer2,
        })
        
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
    
    def getWeights(self, dim, var_name):
        
        return tf.Variable(tf.truncated_normal(shape=dim, mean=0.0, stddev=1.0, dtype=tf.float32), name=var_name)
    
    def getBias(self, dim, var_name):
        
        return tf.Variable(tf.ones(shape=dim, dtype=tf.float32), name=var_name)
        
    def train(self, train_features, train_targets, validation_features, validation_targets):
        
        #forward pass
        hidden_layer1 = tf.add(tf.matmul(self.inputs, self.weights_input_to_hidden), self.bias_hidden_layer1)
        hidden_layer1 = tf.nn.relu(hidden_layer1)
        hidden_layer2 = tf.add(tf.matmul(hidden_layer1, self.weights_hidden_to_hidden), self.bias_hidden_layer2)
        hidden_layer2 = tf.nn.relu(hidden_layer2)
        logits = tf.matmul(hidden_layer2, self.weights_hidden_to_output)
        
        #loss function - softmax with cross entropy
#         softmax = tf.nn.softmax(logits)
#         softmax_log = tf.log(softmax)
#         cross_entropy = tf.multiply(self.outputs, softmax_log)
#         loss = -tf.reduce_sum(cross_entropy)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.outputs))
        
        #optimization function
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.learning_rate).minimize(loss)
        
        #accuracy
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(self.outputs, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        #initializing the variables
        init = tf.global_variables_initializer()
        save_file = "./model.ckpt"
        
        with tf.Session() as sess:
            sess.run(init)
            
            num_records = len(train_features)
            for epoch in range(self.epochs):
                for ii in range(0, num_records, self.batch_size):
                    batch_features = train_features[ii: ii + self.batch_size]
                    batch_targets = train_targets[ii: ii + self.batch_size]
                    feed_dict = {
                        self.inputs : batch_features,
                        self.outputs : batch_targets
                    }
                    sess.run(optimizer, feed_dict=feed_dict)
                    
                if epoch % 2 == 0:
                    validation_accuracy = sess.run(accuracy, feed_dict={
                        self.inputs : validation_features,
                        self.outputs : validation_targets
                    })
                    print("Epoch {:<3} Loss {} Validation Accuracy {:0.3f}".format(epoch, loss, validation_accuracy))
            
            saver.save(sess, save_file)
            print("Model is trained and saved to disk")
            
    def test(test_images, test_labels):
        
        save_file = "./model.ckpt"
        with tf.Session() as sess:
            saver.restore(sess, save_file)
            test_accuracy = sess.run(accuracy, feed_dict={
                self.inputs : test_images, 
                self.labels : test_labels
            })
        print("Test Accuracy {0}".format(test_accuracy))

In [67]:
network = NeuralNetwork(
    num_inputs = 3072,
    num_outputs = 10,
    num_hidden_layer1 = 256,
    num_hidden_layer2 = 64,
    learning_rate = 0.1,
    epochs = 100,
    batch_size = 128
)

In [68]:
network.train(train_images, train_labels, val_images, val_labels)

Epoch 0   Loss Tensor("Mean:0", shape=(), dtype=float32) Validation Accuracy 0.099
Epoch 2   Loss Tensor("Mean:0", shape=(), dtype=float32) Validation Accuracy 0.099


KeyboardInterrupt: 